# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Dantri - BTVN)**

> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```

Cũng giống như VNExpress - Demo


Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ trang báo của Việt Nam.
- Thu thập dữ liệu (bài báo) từ các trang báo của Việt Nam để làm dữ liệu cho các bước xử lý sau.

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
    {
        "url": "https://dantri.com.vn/an-sinh/khong-nen-giam-che-do-om-dau-...",
        "title": "Không nên giảm chế độ ốm đau của người lao động",
        "description": "Chủ tịch công đoàn công ty Intel Product đề nghị không nên ...",
        "content": "[Giảm quyền lợi chế độ ốm đau ảnh hưởng ...",
        "metadata": {
            "cat": "AN SINH",
            "subcat": "DÂN SINH",
            "published_date": 1716159600,
            "author": "Tùng Nguyên"
        },
    },
    ```

# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [ ]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

In [ ]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir -p data

In [ ]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [ ]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [ ]:
MAGAZINE_NAME = "dantri"
HOME_PAGE = "https://dantri.com.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [ ]:
# Choose the menu
#### CODE
####

In [ ]:
# Lấy hết tất cả thể loại từ đây
cats = []

#### CODE
####

In [ ]:
len(cats)

In [ ]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt tham số

In [ ]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 300 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/dantri_url.json"

# Một số thể loại không quan tâm
EXCLUDING_CATEGORIES = ["NHÂN ÁI"] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm

In [ ]:
#### CODE
#### ...

#### Chạy thật

In [ ]:
#### CODE
#### ...

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [ ]:
# Filepath cho cái trước
FILE_URL_PATH = "data/dantri_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/dantri"
#
!mkdir -p $DATA_FOLDER_OUTPUT

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [ ]:
# Đọc file url
with open(FILE_URL_PATH, "r") as fIn:
    url_data = json.load(fIn)

len(url_data)

17

#### Chạy thử nghiệm

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm
SAMPLE_ARTICLE_URLS = [
    "https://dantri.com.vn/xa-hoi/hang-van-du-khach-o-lai-tren-cac-dao-quang-ninh-hai-phong-cho-bao-tan-20240722185725192.htm",
    "https://dantri.com.vn/xa-hoi/nhieu-du-khach-ngoi-thung-o-to-ban-tai-luu-thong-tren-duong-pho-sam-son-20240722163805286.htm",
    "https://dantri.com.vn/the-gioi/ong-trump-doi-boi-thuong-sau-khi-tong-thong-biden-dung-tranh-cu-20240722215254170.htm",
    "https://dantri.com.vn/lao-dong-viec-lam/lao-dong-viet-ve-chan-dung-bac-ho-tong-bi-thu-nguyen-phu-trong-bang-2-tay-20240721101224503.htm",
    "https://dantri.com.vn/khoa-hoc-cong-nghe/chu-cho-lac-nha-dung-dam-mua-nhieu-ngay-cho-chu-den-don-20240722153845731.htm",
    "https://dantri.com.vn/giai-tri/muon-kiep-nhan-sinh-cua-gs-john-vu-nguyen-phong-sap-phat-hanh-20200430134253084.htm" # Author
    # "https://dantri.com.vn/xa-hoi/nguoi-dan-tphcm-xep-hang-thap-huong-tuong-nho-tong-bi-thu-nguyen-phu-trong-20240722193718325.htm" # we ignore this one (emagazine), maybe extensions in the future if you want handle indepedently,
    # "https://dantri.com.vn/xa-hoi/toan-van-bai-viet-khoi-day-niem-tu-hao-ve-dang-cua-tong-bi-thu-20240131111512837.htm" # similar reason like above
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-1]

In [ ]:
driver.get(SAMPLE_ARTICLE_URL)

In [ ]:
# We ignore emagazine for now ..

is_emagazine = False
try:
    driver.find_element(by=By.CLASS_NAME, value="emagazine")
except NoSuchElementException:
    is_emagazine = True

is_emagazine

True

In [ ]:
# Tìm kiếm title

#### CODE
####

In [ ]:
# Tìm kiếm description

#### CODE
####

# Bỏ (dân trí) ở đầu
DANTRI_PHRASE = "(Dân trí) - "
if DANTRI_PHRASE in description:
    description = description.replace(DANTRI_PHRASE, "")

description

In [ ]:
# Thu thập thể loại
#### CODE
####

('GIẢI TRÍ', 'HẠT GIỐNG TÂM HỒN')

In [ ]:
# Thu thập ngày
#### CODE
####

'2020-05-01 09:58'

In [ ]:
# Thu thập tác giả

author = "Unknown"
try:
    #### CODE
    ####
except:
    pass

author

'Unknown'

In [ ]:
# Tìm kiếm contents
#### CODE
####

In [ ]:
# Thu thập contents

contents = []

for idx, child in enumerate(children):
    text = child.text.strip()
    # right align
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
        # If center
        #### CODE
        ####
    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        #### CODE
        ####
    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [ ]:
contents

In [ ]:
driver.close()

#### Chạy thật

In [ ]:

# Class Exception for ignore emagazine
class MagazineArticleException(Exception):
    pass

def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)


    # Tạm thời bỏ qua emagazine
    is_emagazine = True
    try:
        driver.find_element(by=By.CLASS_NAME, value="emagazine")
    except NoSuchElementException:
        is_emagazine = False

    if is_emagazine:
        raise MagazineArticleException("We ignore MagazineArticle")

    # Thu thập title
    #### CODE
    ####

    # Thu thập description
    #### CODE
    ####

    # Thu thập thể loại
    #### CODE
    ####

    # Thu thập published date
    #### CODE
    ####

    # Thu thập người viết
    #### CODE
    ####

    # Thu thập content bài báo
    #### CODE
    ####

    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [ ]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        # NoSuchElementException
        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue

        except MagazineArticleException as e:
            continue


    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w") as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

In [ ]:
# Xem 1 sample
cat_data[-1]

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB